In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [3]:
%pip install --pre -U langchain


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:

import langchain
print(f"LangChain version: {langchain.__version__}")

LangChain version: 1.0.0a13


In [4]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.messages import ToolMessage
from langchain_core.tools import tool
from pydantic import BaseModel, Field
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage
from langsmith import traceable
import base64
import httpx

# create_agent

In [ ]:

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    model="anthropic:claude-3-7-sonnet-latest",
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

## Additional fields in inputs renders weird 
- Doesnt have the "Additional fields" callout like the output does
- Shows up BEFORE the list of messages (should be after)

# init_chat_model

In [5]:
llm_oai = init_chat_model("openai:gpt-4.1")
llm_oai.invoke("Why do parrots talk?")

model_oai = init_chat_model("openai:gpt-4.1")

In [6]:
llm_oai_completions = init_chat_model("openai:gpt-4.1", use_responses_api=False)
llm_oai_completions.invoke("Why do parrots talk?")

AIMessage(content='Parrots "talk" because they are highly intelligent and social birds capable of **vocal mimicry**—the ability to imitate sounds, including human speech. Here’s a more detailed explanation of why parrots talk:\n\n### 1. **Social Interaction**\nIn the wild, parrots use a variety of sounds to communicate with their flock: warning calls, mating calls, signaling locations, forming bonds, etc. **When kept as pets, humans become their “flock,”** so parrots try to communicate with them using the sounds they hear most—often human speech.\n\n### 2. **Imitation and Learning**\nParrots have an extraordinary ability to mimic sounds because of a unique organ in their throat called the **syrinx**. Their brains are also wired for learning and copying. When they hear frequently repeated words or phrases, especially those said with enthusiasm or emotion, they’re more likely to memorize and repeat them.\n\n### 3. **Attention and Reward**\nParrots quickly learn that saying certain words 

In [9]:
llm_anthropic = init_chat_model("anthropic:claude-3-7-sonnet-latest")
model_anthropic = init_chat_model("anthropic:claude-3-7-sonnet-latest")

llm_anthropic.invoke("Why do parrots talk?")

AIMessage(content='Parrots don\'t truly "talk" in the human sense - they mimic sounds they hear in their environment. This ability evolved as a social adaptation that helps them identify with their flock and establish territory in the wild.\n\nIn captivity, parrots often bond with humans as their flock, so they mimic human speech. Their vocal abilities are possible because they have a specialized vocal organ called the syrinx and well-developed brain regions for vocal learning. However, while parrots can associate some words with meanings (like "treat" with food), they don\'t understand grammar or language the way humans do.', additional_kwargs={}, response_metadata={'id': 'msg_01P3gQjMyJMvcMGL3DXopqBj', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 14, 'output_tokens': 13

In [ ]:
inputs = {
  "messages": [
    {
      "id": "msg_123",
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What breed is this dog?"
        },
        {
          "type": "image",
          "source_type": "url",
          "url": "https://fastly.picsum.photos/id/237/200/300.jpg?hmac=TmmQSbShHz9CdQm0NkEjx1Dyh_Y984R9LpNrpvH2D_U",
          "mime_type": "image/jpeg"
        }
      ]
    }
  ]
}

llm.invoke(inputs["messages"])


# model_with_tools

In [10]:
@tool
def get_weather(location: str) -> str:
    """Get the weather at a location."""
    return f"It's sunny in {location}."


model_with_tools = llm_oai.bind_tools([get_weather])

response = model_with_tools.invoke("What's the weather like in Boston?")

## Output preview for oai is wrong. Shows kwargs.additional_kwargs.refusal": null but should be tool_calls.args
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%2222select%22%3Atrue%2C%22id%22%3Atrue%2C%22status%22%3Atrue%2C%22name%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22start_time%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Atrue%2C%22last_queued_at%22%3Atrue%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Atrue%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22feedback_stats%22%3Atrue%2C%22reference_example_id%22%3Atrue%2C%22actions%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&peek=8745caa4-9ae5-4bad-adda-1c4af9908577&peeked_trace=8745caa4-9ae5-4bad-adda-1c4af9908577

In [ ]:
@tool
def get_weather(location: str) -> str:
    """Get the weather at a location."""
    return f"It's sunny in {location}."


model_with_tools = llm_anth.bind_tools([get_weather])

response = model_with_tools.invoke("What's the weather like in Boston?")

## Output preview would be better as tool_calls.args
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%2222select%22%3Atrue%2C%22id%22%3Atrue%2C%22status%22%3Atrue%2C%22name%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22start_time%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Atrue%2C%22last_queued_at%22%3Atrue%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Atrue%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22feedback_stats%22%3Atrue%2C%22reference_example_id%22%3Atrue%2C%22actions%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&peek=40bab8bc-a4a6-40ff-9c83-89a5444a60d0&peeked_trace=40bab8bc-a4a6-40ff-9c83-89a5444a60d0

# structured outputs

In [13]:
class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

model_with_structure = llm_oai.with_structured_output(Movie, include_raw=True)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)  # Movie(title="Inception", year=2010, director="Christopher Nolan", rating=8.8)

{'raw': AIMessage(content='{"title":"Inception","year":2010,"director":"Christopher Nolan","rating":8.8}', additional_kwargs={'parsed': Movie(title='Inception', year=2010, director='Christopher Nolan', rating=8.8), 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 126, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_e24a1fec47', 'id': 'chatcmpl-CQKIYaAlVDO7QuZzF8jhRNAShYV8a', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--20152797-fdc1-4770-9047-7cddc7d7b137-0', usage_metadata={'input_tokens': 126, 'output_tokens': 22, 'total_tokens': 148, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 

In [14]:
class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

model_with_structure = llm_anthropic.with_structured_output(Movie, include_raw=True)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)  # Movie(title="Inception", year=2010, director="Christopher Nolan", rating=8.8)

{'raw': AIMessage(content=[{'id': 'toolu_01WFB1Qy828UkqSbntiQCU3W', 'input': {'title': 'Inception', 'year': 2010, 'director': 'Christopher Nolan', 'rating': 8.8}, 'name': 'Movie', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01NnqzUHGgL3inrzTkQEEEFL', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 465, 'output_tokens': 91, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-7-sonnet-20250219', 'model_provider': 'anthropic'}, id='lc_run--8e5f3133-8445-4a24-923c-602078823c67-0', tool_calls=[{'name': 'Movie', 'args': {'title': 'Inception', 'year': 2010, 'director': 'Christopher Nolan', 'rating': 8.8}, 'id': 'toolu_01WFB1Qy828UkqSbntiQCU3W', 'type': 'tool_call'}], usage_metadata={'input_tokens': 465, 'output_tokens': 91, 't

## Output preview currently shows empty AI message. Should show the array in "parsed"
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%2222select%22%3Atrue%2C%22id%22%3Atrue%2C%22status%22%3Atrue%2C%22name%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22start_time%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Atrue%2C%22last_queued_at%22%3Atrue%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Atrue%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22feedback_stats%22%3Atrue%2C%22reference_example_id%22%3Atrue%2C%22actions%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&peek=94e62bba-30b4-401f-b50d-e402e42d22af&peeked_trace=94e62bba-30b4-401f-b50d-e402e42d22af


(the output preview rendering worked for openai: https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%2222select%22%3Atrue%2C%22id%22%3Atrue%2C%22status%22%3Atrue%2C%22name%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22start_time%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Atrue%2C%22last_queued_at%22%3Atrue%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Atrue%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22feedback_stats%22%3Atrue%2C%22reference_example_id%22%3Atrue%2C%22actions%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&peek=57ec9890-6a42-4001-b00d-bb90b07667db&peeked_trace=57ec9890-6a42-4001-b00d-bb90b07667db)

# tools and tool messages

In [15]:
# After a model makes a tool call
ai_message = AIMessage(
    content=[],
    tool_calls=[{
        "name": "get_weather",
        "args": {"location": "San Francisco"},
        "id": "call_123"
    }]
)

# Execute tool and create result message
weather_result = "Sunny, 72°F"
tool_message = ToolMessage(
    content=weather_result,
    tool_call_id="call_123"  # Must match the call ID
)

# Continue conversation
messages = [
    HumanMessage("What's the weather in San Francisco?"),
    ai_message,  # Model's tool call
    tool_message,  # Tool execution result
]
response = model_oai.invoke(messages)  # Model processes the result

In [16]:
# After a model makes a tool call
ai_message = AIMessage(
    content=[],
    tool_calls=[{
        "name": "get_weather",
        "args": {"location": "San Francisco"},
        "id": "call_123"
    }]
)

# Execute tool and create result message
weather_result = "Sunny, 72°F"
tool_message = ToolMessage(
    content=weather_result,
    tool_call_id="call_123"  # Must match the call ID
)

# Continue conversation
messages = [
    HumanMessage("What's the weather in San Francisco?"),
    ai_message,  # Model's tool call
    tool_message,  # Tool execution result
]
response = model_anthropic.invoke(messages)  # Model processes the result

# Content Blocks 

## Reasoning and image inputs 

In [17]:
from langchain_core.messages import HumanMessage

# String content
messages = [HumanMessage(content_blocks=[
    {"type": "text", "text": "Hello, how are you?"},
    {"type": "image", "url": "https://fastly.picsum.photos/id/237/200/300.jpg?hmac=TmmQSbShHz9CdQm0NkEjx1Dyh_Y984R9LpNrpvH2D_U"},
])] 

llm.invoke(messages)

AIMessage(content="Hello! I'm doing well, thank you for asking. \n\nI see you've shared an image of an adorable black Labrador puppy! The pup has a shiny black coat and those sweet, expressive eyes looking up at the camera. It appears to be resting on what looks like a wooden surface, possibly a deck or floor. Black Labs are known for being intelligent, friendly, and loyal companions. They're quite popular family dogs and are often great with children and other pets.\n\nIs this your puppy? They're absolutely precious!", additional_kwargs={}, response_metadata={'id': 'msg_01H34fDod332Mih4413F5fxz', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 104, 'output_tokens': 119, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-7-sonnet-20250219', 'model_

In [19]:
reasoning = {
    "effort": "medium",  # 'low', 'medium', or 'high'
    "summary": "auto",  # 'detailed', 'auto', or None
}

thinking={"type": "enabled", "budget_tokens": 2000}
model_oai_content_blocks = init_chat_model("openai:gpt-5-nano", output_version="v1", reasoning=reasoning)
#model_anthropic_content_blocks = init_chat_model("anthropic:claude-3-7-sonnet-latest", output_version="v1", thinking=thinking)
model_anthropic_content_blocks = init_chat_model("anthropic:claude-3-7-sonnet-latest", output_version="v1")

model_oai_completions_content_blocks = init_chat_model("openai:gpt-4.1", use_responses_api=False, output_version="v1")




In [23]:
inputs = [HumanMessage(content_blocks=[
    {"type": "text", "text": "What is in this image?"},
    {"type": "image", "url": "https://fastly.picsum.photos/id/237/200/300.jpg?hmac=TmmQSbShHz9CdQm0NkEjx1Dyh_Y984R9LpNrpvH2D_U"},
])] 

response = model_oai_content_blocks.invoke(inputs)

print(response)

content=[{'id': 'rs_0f50d63073ee0c9f0068eedfe4b98881a2962be71ac0b051fa', 'type': 'reasoning'}, {'type': 'text', 'text': 'A black dog (likely a puppy) lying on a wooden floor, looking up at the camera with big eyes.', 'annotations': [], 'id': 'msg_0f50d63073ee0c9f0068eedfe7579881a29b9673784fa9fcb1'}] additional_kwargs={} response_metadata={'id': 'resp_0f50d63073ee0c9f0068eedfe3991481a2a6753697258e3fd8', 'created_at': 1760485348.0, 'metadata': {}, 'model': 'gpt-5-nano-2025-08-07', 'object': 'response', 'service_tier': 'default', 'status': 'completed', 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'output_version': 'v1'} id='resp_0f50d63073ee0c9f0068eedfe3991481a2a6753697258e3fd8' usage_metadata={'input_tokens': 118, 'output_tokens': 349, 'total_tokens': 467, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 320}}


In [12]:
response = model_oai_completions_content_blocks.invoke(inputs)

print(response)


content=[{'type': 'text', 'text': 'This image shows a black Labrador Retriever puppy lying on a wooden surface. The puppy is looking up toward the camera with a sweet and curious expression.'}] additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 268, 'total_tokens': 297, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_e24a1fec47', 'id': 'chatcmpl-CQiemg2DbKIYvcuQFML8DDBmoJw8W', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None, 'output_version': 'v1'} id='lc_run--c8118215-ed51-4813-a731-b7cd62818e49-0' usage_metadata={'input_tokens': 268, 'output_tokens': 29, 'total_tokens': 297, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio

## Reasoning doesnt render (openai)
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%2222select%22%3Atrue%2C%22id%22%3Atrue%2C%22status%22%3Atrue%2C%22name%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22start_time%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Atrue%2C%22last_queued_at%22%3Atrue%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Atrue%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22feedback_stats%22%3Atrue%2C%22reference_example_id%22%3Atrue%2C%22actions%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&peek=0d86fa2a-ac00-4741-bab9-31be10a726c7&peeked_trace=0d86fa2a-ac00-4741-bab9-31be10a726c7

In [30]:
inputs = [HumanMessage(content_blocks=[
    {"type": "text", "text": "What is in this image?"},
    {"type": "image", "url": "https://fastly.picsum.photos/id/237/200/300.jpg?hmac=TmmQSbShHz9CdQm0NkEjx1Dyh_Y984R9LpNrpvH2D_U"},
])] 

response = model_anthropic_content_blocks.invoke(inputs)

print(response)

content=[{'type': 'reasoning', 'reasoning': "The image shows a black puppy, likely a Labrador Retriever or a similar breed, sitting or lying on what appears to be a wooden floor or surface. The puppy has a shiny black coat and dark eyes that give it an adorable, slightly sad expression. The puppy is looking directly at the camera. This is a close-up photo focusing on the puppy's face and upper body.\n\nI should describe what I see in the image, which is simply a black puppy dog on a wooden surface. There are no human faces in this image, so I can respond normally.", 'extras': {'signature': 'ErUBCkYICBgCIkDvgR0FsxpIUwDdxBUrHWHCMnqedR1+Sx+HI166a6XwUeE7dZrgOygjG+RJXPepCA/uK+ZjakVJ//3UPyN24ZAtEgySuKtxhkaK3xMOLAAaDAJksA0b70fvhKpW6iIw6W4ZUI/7tATItObcgRI8SP/Jqf02AycHxXO+S+IVvoM2EvSOIIdfPegqEhHxmmKnKh3ihgt0kErAKbSugqjPLP6GWtFJrGwgm2t2ACdQlhgC'}}, {'type': 'text', 'text': "This image shows an adorable black puppy lying on what appears to be a wooden surface. It has a shiny black coat and expres

## Reasoning doesnt render (anthropic)
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%2222select%22%3Atrue%2C%22id%22%3Atrue%2C%22status%22%3Atrue%2C%22name%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22start_time%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Atrue%2C%22last_queued_at%22%3Atrue%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Atrue%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22feedback_stats%22%3Atrue%2C%22reference_example_id%22%3Atrue%2C%22actions%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D

## PDF inputs 

In [22]:
inputs = [HumanMessage(content_blocks=[
    {"type": "text", "text": "What is in this image?"},
    {"type": "file", "url": "https://pdfobject.com/pdf/sample.pdf"},
])] 

response = model_oai_content_blocks.invoke(inputs)

print(response)

content=[{'id': 'rs_0dc5ca941f66819f0068eedf8f2c7c81918381ffd97aa3c318', 'type': 'reasoning', 'reasoning': '**Interpreting image content**\n\nI see the user asked, "What is in this image?" They provided some sample text that looks like it\'s from a PDF. The image capabilities are enabled, so maybe there\'s an image attached, but I only see text. I should respond by describing it as a sample PDF with Lorem ipsum text. The user might expect a brief description like, "It\'s a sample PDF document containing placeholder text with repeated Latin phrases." I\'ll keep it concise and clear!'}, {'id': 'rs_0dc5ca941f66819f0068eedf8f2c7c81918381ffd97aa3c318', 'type': 'reasoning', 'reasoning': "**Describing the PDF sample**\n\nI need to answer that it's a simple PDF sample file with Lorem ipsum text, showing long paragraphs of placeholder text. There's no meaningful content here; it’s just standard filler. I could also ask the user if they want me to extract topics, but since it’s all filler, there

### PDF doesnt render 
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%2222select%22%3Atrue%2C%22id%22%3Atrue%2C%22status%22%3Atrue%2C%22name%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22start_time%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Atrue%2C%22last_queued_at%22%3Atrue%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Atrue%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22feedback_stats%22%3Atrue%2C%22reference_example_id%22%3Atrue%2C%22actions%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&runtab=0&peek=a4e20cab-9cf9-41a3-ae5d-ecf4c7a97f21&peeked_trace=a4e20cab-9cf9-41a3-ae5d-ecf4c7a97f21

In [ ]:
inputs = [HumanMessage(content_blocks=[
    {"type": "text", "text": "What is in this pdf?"},
    {"type": "file", "url": "https://pdfobject.com/pdf/sample.pdf"},
])] 

response = model_anthropic_content_blocks.invoke(inputs)

print(response)

In [25]:
pdf_url = "https://pdfobject.com/pdf/sample.pdf"
pdf_data = base64.b64encode(httpx.get(pdf_url).content).decode("utf-8")

inputs = [HumanMessage(content_blocks=[
    {"type": "text", "text": "What is in this pdf?"},
    {"type": "file", "base64": pdf_data, "mime_type": "application/pdf"},
])] 

response = model_oai_completions_content_blocks.invoke(inputs)

print(response)

/Users/tanushreesharma/Documents/Projects/ls-tracing-testing/.venv/lib/python3.13/site-packages/langchain_core/messages/block_translators/openai.py:82: UserWarning: OpenAI may require a filename for file uploads. Specify a filename in the content block, e.g.: {'type': 'file', 'mime_type': '...', 'base64': '...', 'filename': 'my-file.pdf'}
  warnings.warn(


BadRequestError: Error code: 400 - {'error': {'message': "Missing required parameter: 'messages[0].content[1].file.file_id'.", 'type': 'invalid_request_error', 'param': 'messages[0].content[1].file.file_id', 'code': 'missing_required_parameter'}}

In [27]:
inputs = [HumanMessage(content_blocks=[
    {"type": "text", "text": "What is in this image?"},
    {"type": "file", "url": "https://pdfobject.com/pdf/sample.pdf"},
])] 

response = model_anthropic_content_blocks.invoke(inputs)

print(response)

content=[{'type': 'text', 'text': 'This is a PDF document titled "Sample PDF" with the subtitle "This is a simple PDF file. Fun fun fun." The document contains several paragraphs of Lorem ipsum placeholder text, which is commonly used as filler text in publishing and graphic design. The text is formatted in a standard document layout with paragraphs of Latin-like text that doesn\'t convey any specific meaning. It appears to be a demonstration or sample PDF file that might be used for testing purposes.'}] additional_kwargs={} response_metadata={'id': 'msg_01XwDkkKaJf6i5SBXZ2M2bVt', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 2679, 'output_tokens': 100, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-7-sonnet-20250219', 'model_provider': 'anth

### PDF doesnt render 

https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%2222select%22%3Atrue%2C%22id%22%3Atrue%2C%22status%22%3Atrue%2C%22name%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22start_time%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Atrue%2C%22last_queued_at%22%3Atrue%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Atrue%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22feedback_stats%22%3Atrue%2C%22reference_example_id%22%3Atrue%2C%22actions%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&runtab=0&peek=db852d70-589d-426e-b5fe-442d945d7f5a&peeked_trace=db852d70-589d-426e-b5fe-442d945d7f5a

In [30]:

model_oai_content_blocks_audio = init_chat_model("openai:gpt-4o-audio-preview", output_version="v1")


pdf_url = "https://people.math.sc.edu/Burkardt/data/wav/thermo.wav"
pdf_data = base64.b64encode(httpx.get(pdf_url).content).decode("utf-8")

inputs = [HumanMessage(content_blocks=[
    {"type": "audio", "base64": pdf_data, "mime_type": "audio/wav"},
])] 

response = model_oai_content_blocks_audio.invoke(inputs)

print(response)

content=[{'type': 'text', 'text': "That’s a great reference! It sounds like you're channeling a famous quote from a popular animated show. The laws of thermodynamics, of course, are the pillars of understanding energy and heat in any system. \n\n1. Energy cannot be created or destroyed – only transformed.\n2. In isolated systems, entropy (disorder) tends to increase.\n3. As temperature approaches absolute zero, the entropy of a system approaches a constant minimum.\n\nTogether, they help us make sense of everything from engines to refrigerators to stars.\n\nSo rest assured—thermodynamics rules the house!"}] additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 115, 'prompt_tokens': 44, 'total_tokens': 159, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0, 'text_tokens': 115}, 'prompt_tokens_details': {'audio_tokens': 33, 'cached_tokens': 0, 'text_tokens': 11, 'i

## we don't show the audio file in pretty rendering
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%2222select%22%3Atrue%2C%22id%22%3Atrue%2C%22status%22%3Atrue%2C%22name%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22start_time%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Atrue%2C%22last_queued_at%22%3Atrue%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Atrue%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22feedback_stats%22%3Atrue%2C%22reference_example_id%22%3Atrue%2C%22actions%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&runtab=0&peek=3a2cf2c3-e96d-4658-921d-738cb313ed20&peeked_trace=3a2cf2c3-e96d-4658-921d-738cb313ed20

## streaming

In [13]:
for chunk in model_oai_content_blocks.stream("Why do parrots have colorful feathers?"):
    print(chunk.text, end="|", flush=True)

||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||Par|rots| are| colorful| mainly| because| of| how| their| feathers| get| color| and| what| those| colors| are| used| for|.

|How| the| colors| come| about|
|-| Pig|ments|:
| | -| Car|ot|eno|ids| (|from| diet|)| give| reds|,| oranges|,| yell|ows|.
| | -| Mel|an|ins| produce| blacks| and| brow|ns|.
| | -| P

In [14]:
for chunk in model_oai_completions_content_blocks.stream("Why do parrots have colorful feathers?"):
    print(chunk.text, end="|", flush=True)

|Par|rots| have| colorful| feathers| for| several| reasons| related| to| **|e|volution|,| communication|,| and| survival|**|:

|1|.| **|M|ating| and| Attraction|**|:| Bright|ly| colored| feathers| help| parro|ts| attract| mates|.| Many| par|rot| species| rely| on| visual| signals| for| court|ship|,| so| vibrant| colors| indicate| health|,| genetic| fitness|,| and| suitability| as| a| breeding| partner|.

|2|.| **|Species| and| Individual| Recognition|**|:| The| variety| of| colors| and| patterns| helps| parro|ts| recognize| members| of| their| own| species| and| sometimes| even| distinguish| individuals| within| a| group|.

|3|.| **|Cam|ouflage|**|:| Surprisingly|,| vivid| feather| colors| can| also| help| parro|ts| blend| into| their| natural| habitats|,| such| as| the| bright| greens|,| reds|,| and| yell|ows| found| in| rainforest| can|opies| and| flowers|.| These| colors| may| break| up| the| outline| of| the| bird|,| making| it| harder| for| predators| to| spot| them| among| the| f

In [15]:
for chunk in model_anthropic_content_blocks.stream("Why do parrots have colorful feathers?"):
    print(chunk.text, end="|", flush=True)

|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||# Why Parrots Have| Colorful Feathers|

Parrots|'| vib|rant pl|umage serves| several important biological purposes|:

## Sexual| Selection|
The| bright colors often| play| a crucial| role in attract|ing mates. More| vib|rant individuals| may signal| better| health an|d genetic quality| to potential partners.|

## Species Recognition|
Distinctive color patterns help| parrots identify members| of their own species in| dense| forests,| which| is essential| for social interaction| and finding compatible| mates.

##| Camouflage|
Though| it| might seem counterintu|itive, bright colors can| provide| effective camouflage| in tropical| forests| full| of colorful fruits an|d flowers.|

## Social| Communication|
Within| parrot communities|, color| variations| can| signal age|, status, an|d health to other| members| of| their| f|lock.|

Inter|estingly, parr|ots produce

In [ ]:
for chunk in model_oai).stream("Why do parrots have colorful feathers?"):
    print(chunk.text, end="|", flush=True)

##  structured output

In [16]:
class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

model_with_structure = model_oai_content_blocks.with_structured_output(Movie, include_raw=True)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)  # Movie(title="Inception", year=2010, director="Christopher Nolan", rating=8.8)

{'raw': AIMessage(content=[{'id': 'rs_0e2b00121d7abfd30068eedd2ba14481a2a2bb1a7f0cc9b5d0', 'type': 'reasoning', 'reasoning': '**Structuring movie details**\n\nThe user wants details about the movie Inception. I should respond in the structured "Movie" JSON format, which includes fields like title, year, director, and rating. Since rating could be around 8.8, I want to confirm the format before finalizing it. The instructions mention the JSON schema, so a structured response would likely be expected. I’ll make sure it aligns with the schema while keeping it clear and easy to understand!'}, {'id': 'rs_0e2b00121d7abfd30068eedd2ba14481a2a2bb1a7f0cc9b5d0', 'type': 'reasoning', 'reasoning': '**Defining JSON output format**\n\nThe instruction clarifies that the output must be parsed by code or shown in an app, which may not support special formatting like bullets. I need to follow the "Movie" JSON schema precisely, including keys such as title, year, director, and rating. For instance, I’d re

## we show too much in the pretty rendering 

Would remove all fields under "response_metadata", "tool_calls", "usage_metadata"

https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%2222select%22%3Atrue%2C%22id%22%3Atrue%2C%22status%22%3Atrue%2C%22name%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22start_time%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Atrue%2C%22last_queued_at%22%3Atrue%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Atrue%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22feedback_stats%22%3Atrue%2C%22reference_example_id%22%3Atrue%2C%22actions%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&runtab=0&peek=70840561-16d0-4de5-a6b1-21e39d5de2d9

In [17]:
class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

model_with_structure = model_oai_completions_content_blocks.with_structured_output(Movie, include_raw=True)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)  # Movie(title="Inception", year=2010, director="Christopher Nolan", rating=8.8)

{'raw': AIMessage(content=[{'type': 'text', 'text': '{"title":"Inception","year":2010,"director":"Christopher Nolan","rating":8.8}'}], additional_kwargs={'parsed': Movie(title='Inception', year=2010, director='Christopher Nolan', rating=8.8), 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 126, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_e24a1fec47', 'id': 'chatcmpl-CQimmeL9epyYNpLXtlBv219q7vkdC', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None, 'output_version': 'v1'}, id='lc_run--9ace9590-b011-44a8-85d2-b01ee024d813-0', usage_metadata={'input_tokens': 126, 'output_tokens': 22, 'total_tokens': 148, 'input_token_details': {'audio': 0, 'cache_read': 0}

In [20]:
class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

model_with_structure = model_anthropic_content_blocks.with_structured_output(Movie, include_raw=True)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)  # Movie(title="Inception", year=2010, director="Christopher Nolan", rating=8.8)

{'raw': AIMessage(content=[{'type': 'tool_call', 'name': 'Movie', 'args': {'title': 'Inception', 'year': 2010, 'director': 'Christopher Nolan', 'rating': 8.8}, 'id': 'toolu_016EAVVcgrwEpaEu3uND8bUT'}], additional_kwargs={}, response_metadata={'id': 'msg_01WAHxEwpLhye5gCBrXCASR5', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 465, 'output_tokens': 91, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-7-sonnet-20250219', 'model_provider': 'anthropic', 'output_version': 'v1'}, id='lc_run--f7899a2f-8f5f-4118-b705-b122dbb608e9-0', tool_calls=[{'name': 'Movie', 'args': {'title': 'Inception', 'year': 2010, 'director': 'Christopher Nolan', 'rating': 8.8}, 'id': 'toolu_016EAVVcgrwEpaEu3uND8bUT', 'type': 'tool_call'}], usage_metadata={'input_tokens': 465,

## we show too much in the pretty rendering 

Would remove all fields under "response_metadata", "tool_calls", "usage_metadata"

https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%2222select%22%3Atrue%2C%22id%22%3Atrue%2C%22status%22%3Atrue%2C%22name%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22start_time%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Atrue%2C%22last_queued_at%22%3Atrue%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Atrue%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22feedback_stats%22%3Atrue%2C%22reference_example_id%22%3Atrue%2C%22actions%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&runtab=0&peek=7183b1d4-da3f-4baa-a614-1c3d9050b2fe&peeked_trace=7183b1d4-da3f-4baa-a614-1c3d9050b2fe

## server-side tools

In [ ]:
# openai: web search
model_oai_content_blocks_tools = init_chat_model("openai:gpt-4.1-mini", output_version="v1")

tool = {"type": "web_search"}
model_with_tools = model_oai_content_blocks_tools.bind_tools([tool])

response = model_with_tools.invoke("What was a positive news story from today?")
response.content_blocks

[{'type': 'server_tool_call',
  'name': 'web_search',
  'args': {'query': 'positive news story today', 'type': 'search'},
  'id': 'ws_0465e447a69b454e0068ed78255b60819f9508ba0ef0d9f863'},
 {'type': 'server_tool_result',
  'tool_call_id': 'ws_0465e447a69b454e0068ed78255b60819f9508ba0ef0d9f863',
  'status': 'success'},
 {'type': 'text',
  'text': 'A positive news story from today is the release of all 20 living Israeli hostages held by Hamas in Gaza. This development marks a significant step toward peace, as Israel has agreed to release 250 Palestinian prisoners and over 1,700 detainees in exchange. ([en.wikipedia.org](https://en.wikipedia.org/wiki/2025?utm_source=openai))\n\nAdditionally, in Flowery Branch, Georgia, a high school community came together to support a security guard in financial need by providing her with a new car. This act of kindness was initiated by students and supported by local businesses. ([soundcloud.com](https://soundcloud.com/user-423075597/uplifting-and-771571

## we don't render hyperlinks for citations correctly 
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%2222select%22%3Atrue%2C%22id%22%3Atrue%2C%22status%22%3Atrue%2C%22name%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22start_time%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Atrue%2C%22last_queued_at%22%3Atrue%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Atrue%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22feedback_stats%22%3Atrue%2C%22reference_example_id%22%3Atrue%2C%22actions%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&peek=c35f72c4-d8e8-4463-9482-6f82a13603dd&peeked_trace=c35f72c4-d8e8-4463-9482-6f82a13603dd

In [ ]:
# openai: image generation
tool = {"type": "image_generation", "quality": "low"}

llm_with_tools = model_oai_content_blocks_tools.bind_tools([tool])

ai_message = llm_with_tools.invoke(
    "Draw a picture of a cute fuzzy cat with an umbrella"
)

## we don't render the base64 image output
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%2222select%22%3Atrue%2C%22id%22%3Atrue%2C%22status%22%3Atrue%2C%22name%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22start_time%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Atrue%2C%22last_queued_at%22%3Atrue%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Atrue%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22feedback_stats%22%3Atrue%2C%22reference_example_id%22%3Atrue%2C%22actions%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&peek=074e9352-afc3-4b59-bccb-b6967bec9184&peeked_trace=074e9352-afc3-4b59-bccb-b6967bec9184

In [ ]:
# openai:code interpreter

from langchain_openai import ChatOpenAI

llm_with_tools = model_oai_content_blocks_tools.bind_tools(
    [
        {
            "type": "code_interpreter",
            # Create a new container
            "container": {"type": "auto"},
        }
    ]
)
response = llm_with_tools.invoke(
    "Write and run code to answer the question: what is 3^3?"
)

## Tools appears empty
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%2222select%22%3Atrue%2C%22id%22%3Atrue%2C%22status%22%3Atrue%2C%22name%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22start_time%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Atrue%2C%22last_queued_at%22%3Atrue%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Atrue%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22feedback_stats%22%3Atrue%2C%22reference_example_id%22%3Atrue%2C%22actions%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&peek=17fd9dc7-4630-4112-a378-f28884308cee&peeked_trace=17fd9dc7-4630-4112-a378-f28884308cee

### didnt test openai file search, computer use 

In [39]:
# anthropic: files API

import anthropic

client = anthropic.Anthropic()

file = client.beta.files.upload(
    file=("document.pdf", open("/Users/tanushreesharma/Documents/Projects/ls-tracing-testing/python/document.pdf", "rb"), "application/pdf"),
)
pdf_file_id = file.id

model_anthropic_content_blocks_files = init_chat_model("anthropic:claude-3-7-sonnet-latest", output_version="v1", thinking=thinking, betas=["files-api-2025-04-14"])

input_message = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe this document."},
        {"type": "file", "file_id": pdf_file_id}
    ],
}
model_anthropic_content_blocks_files.invoke([input_message])

AIMessage(content=[{'type': 'reasoning', 'reasoning': 'This document is an announcement or flyer for a workshop event titled "Hands-on Workshop: Evaluating AI in High-Stakes Applications." Let me analyze its key components:\n\n1. Event Details:\n   - Title: "Hands-on Workshop: Evaluating AI in High-Stakes Applications"\n   - Date: September 16, 2025\n   - Time: 8:45-10:15 AM\n   - Location: QED Building, SF\n   - Note: Breakfast provided\n   - Facilitation: BikeShop@MIT as part of the EDGES Summit\n\n2. Purpose Section ("What This Is"):\n   - An invite-only, off-the-record exchange between AI practitioners working with user-facing systems in high-stakes contexts\n   - Focus on settings where AI assists humans in making important decisions\n   - Examples mentioned: complex customer service, financial advisory, legal support, health, career coaching, sales enablement\n   - No pitches or demos—just honest conversations about what works and what doesn\'t\n   - Bringing together four perspe

## this might be an OSS thing, but ideally we trace the file too
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%2222select%22%3Atrue%2C%22id%22%3Atrue%2C%22status%22%3Atrue%2C%22name%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22start_time%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Atrue%2C%22last_queued_at%22%3Atrue%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Atrue%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22feedback_stats%22%3Atrue%2C%22reference_example_id%22%3Atrue%2C%22actions%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&peek=22d55efa-c44f-4459-b398-7fb30df66dfc&peeked_trace=22d55efa-c44f-4459-b398-7fb30df66dfc

In [40]:
# anthropic: citations
from langchain_anthropic import ChatAnthropic

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "document",
                "source": {
                    "type": "text",
                    "media_type": "text/plain",
                    "data": "The grass is green. The sky is blue.",
                },
                "title": "My Document",
                "context": "This is a trustworthy document.",
                "citations": {"enabled": True},
            },
            {"type": "text", "text": "What color is the grass and sky?"},
        ],
    }
]
response = model_oai_content_blocks.invoke(messages)
response.content

[{'id': 'rs_0d9cf690af3d23200068ed7df76510819ca2c9b69a152f9d8e',
  'type': 'reasoning',
  'reasoning': "**Clarifying color question**\n\nThe user asks about the colors of grass and sky, but it's a bit ambiguous. I'm wondering if they mean specific grass and sky or just a general observation. They might have uploaded an image, but their text doesn’t mention it directly. If it is an image, I should analyze it. For now, I think it’s best to give a general answer: grass is typically green, and the sky is blue during the daytime."},
 {'id': 'rs_0d9cf690af3d23200068ed7df76510819ca2c9b69a152f9d8e',
  'type': 'reasoning',
  'reasoning': "**Providing color context**\n\nI also want to mention that grass can vary in color during dawn or dusk and might appear gray in overcast conditions. However, the user's question feels straightforward, so I’ll keep it concise: grass is generally green, and the sky is blue, assuming it’s daytime. It’s good to include that colors can change with sunrise, sunset, 

In [42]:
# anthropic: web search 

tool = {"type": "web_search_20250305", "name": "web_search", "max_uses": 3}
llm_with_tools = model_oai_content_blocks.bind_tools([tool])

response = model_anthropic_content_blocks.invoke("How do I update a web app to TypeScript 5.5?")


In [43]:
# anthropic: fetch
model_anthropic_content_blocks_web_fetch = init_chat_model("anthropic:claude-3-7-sonnet-latest", output_version="v1", thinking=thinking, betas=["web-fetch-2025-09-10"])

tool = {"type": "web_fetch_20250910", "name": "web_fetch", "max_uses": 3}
llm_with_tools = model_anthropic_content_blocks_web_fetch.bind_tools([tool])

response = llm_with_tools.invoke(
    "Please analyze the content at https://example.com/article"
)

### didnt test memory tool, text editor

remote MCP: tested in wrap_anthropic.ipynb

In [47]:
## openai logprobs

model_oai_content_blocks = init_chat_model("gpt-4o", output_version="v1").bind(logprobs=True)

response = model_oai_content_blocks.invoke("Why do parrots talk?")
print(response.response_metadata["logprobs"])



response = model_oai_content_blocks.invoke("Why do parrots talk?")
print(response.response_metadata["logprobs"])


{'content': [{'token': 'Par', 'bytes': [80, 97, 114], 'logprob': -7.896309739408025e-07, 'top_logprobs': []}, {'token': 'rots', 'bytes': [114, 111, 116, 115], 'logprob': -1.1472419600977446e-06, 'top_logprobs': []}, {'token': ' are', 'bytes': [32, 97, 114, 101], 'logprob': -0.1283116638660431, 'top_logprobs': []}, {'token': ' known', 'bytes': [32, 107, 110, 111, 119, 110], 'logprob': -0.04957265406847, 'top_logprobs': []}, {'token': ' for', 'bytes': [32, 102, 111, 114], 'logprob': -0.0021852378267794847, 'top_logprobs': []}, {'token': ' their', 'bytes': [32, 116, 104, 101, 105, 114], 'logprob': -2.1008713702030946e-06, 'top_logprobs': []}, {'token': ' ability', 'bytes': [32, 97, 98, 105, 108, 105, 116, 121], 'logprob': -0.008262669667601585, 'top_logprobs': []}, {'token': ' to', 'bytes': [32, 116, 111], 'logprob': 0.0, 'top_logprobs': []}, {'token': ' mimic', 'bytes': [32, 109, 105, 109, 105, 99], 'logprob': -0.009718899615108967, 'top_logprobs': []}, {'token': ' human', 'bytes': [32, 

## we dont render logprobs

https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%2222select%22%3Atrue%2C%22id%22%3Atrue%2C%22status%22%3Atrue%2C%22name%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22start_time%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Atrue%2C%22last_queued_at%22%3Atrue%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Atrue%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22feedback_stats%22%3Atrue%2C%22reference_example_id%22%3Atrue%2C%22actions%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&peek=05a5b7cc-d554-45de-8505-46174aaabc70&peeked_trace=05a5b7cc-d554-45de-8505-46174aaabc70

# Multi-modal (constructing messages directly)

In [ ]:
#images

image_url = "https://fastly.picsum.photos/id/237/200/300.jpg?hmac=TmmQSbShHz9CdQm0NkEjx1Dyh_Y984R9LpNrpvH2D_U"
image_base64 = base64.standard_b64encode(httpx.get(image_url).content).decode("utf-8")


inputs = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this image."},
        {"type": "image", "url": "https://fastly.picsum.photos/id/237/200/300.jpg?hmac=TmmQSbShHz9CdQm0NkEjx1Dyh_Y984R9LpNrpvH2D_U"},
    ]
}

# From base64 data
output = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this image."},
        {
            "type": "image",
            "base64": image_base64,
            "mime_type": "image/jpeg",
        },
    ]
} 

@traceable(run_type="llm")
def llm_raw(inputs):
    return output


llm_raw(inputs)


## Gap in image rendering 

We don't render images
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%2222select%22%3Atrue%2C%22id%22%3Atrue%2C%22status%22%3Atrue%2C%22name%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22start_time%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Atrue%2C%22last_queued_at%22%3Atrue%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Atrue%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22feedback_stats%22%3Atrue%2C%22reference_example_id%22%3Atrue%2C%22actions%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&peek=7c87a2bd-fa0e-4aa0-b8b3-2719852a8757&peeked_trace=7c87a2bd-fa0e-4aa0-b8b3-2719852a8757

In [ ]:
#pdf

pdf_url = "https://pdfobject.com/pdf/sample.pdf"
pdf_base64 = base64.standard_b64encode(httpx.get(pdf_url).content).decode("utf-8")


inputs = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this document."},
        {"type": "file", "url": "https://pdfobject.com/pdf/sample.pdf"},
    ]
}

# From base64 data
output = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this document."},
        {
            "type": "file",
            "base64": pdf_base64,
            "mime_type": "application/pdf",
        },
    ]
} 



@traceable(run_type="llm")
def llm_raw(inputs):
    return output


llm_raw(inputs)

## Gap in redering PDFs

Same issue with PDFs

https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%2222select%22%3Atrue%2C%22id%22%3Atrue%2C%22status%22%3Atrue%2C%22name%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22start_time%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Atrue%2C%22last_queued_at%22%3Atrue%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Atrue%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22feedback_stats%22%3Atrue%2C%22reference_example_id%22%3Atrue%2C%22actions%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&peek=1955e5c3-b539-4029-8321-25bb25a344b1&peeked_trace=1955e5c3-b539-4029-8321-25bb25a344b1

Not going to try audio, video but likely the same 

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import ToolNode, create_agent
import random

@tool
def fetch_user_data(user_id: str) -> str:
    """Fetch user data from database."""
    if random.random() > 0.7:
        raise ConnectionError("Database connection timeout")
    return f"User {user_id}: John Doe, john@example.com, Active"

@tool
def process_transaction(amount: float, user_id: str) -> str:
    """Process a financial transaction."""
    if amount > 10000:
        raise ValueError(f"Amount {amount} exceeds maximum limit of 10000")
    return f"Processed ${amount} for user {user_id}"

def handle_errors(e: Exception) -> str:
    if isinstance(e, ConnectionError):
        return "The database is currently overloaded, but it is safe to retry. Please try again with the same parameters."
    elif isinstance(e, ValueError):
        return f"Error: {e}. Try to process the transaction in smaller amounts."
    return f"Error: {e}. Please try again."

tool_node = ToolNode(
    tools=[fetch_user_data, process_transaction],
    handle_tool_errors=handle_errors
)

agent = create_agent(
    model=ChatOpenAI(model="gpt-4o"),
    tools=tool_node,
    prompt="You are a financial assistant."
)

agent.invoke({
    "messages": [{"role": "user", "content": "Process a payment of 15000 dollars for user123. Generate a receipt email and address it to the user."}]
})